In [ ]:
# ==========================
# 0. CÀI ĐẶT & IMPORT
# ==========================
# !pip install umap-learn

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
import umap

# Mặc định hiển thị đẹp hơn
plt.rcParams["figure.figsize"] = (12, 5)
plt.rcParams["axes.grid"] = True


In [ ]:
# ==========================
# 1. ĐỌC DỮ LIỆU
# ==========================
# ĐỔI đường dẫn cho đúng file của chị
df = pd.read_csv("../data/interim/rri_hrv_full_quality.csv")

print(df.shape)
df.head()


In [ ]:
# ==========================
# 2. XÁC ĐỊNH CÁC CỘT FEATURE
# ==========================
# Giả sử tên cột RRI_* và HRV_* có prefix rõ ràng
rri_cols = [c for c in df.columns if c.startswith("RRI_")]
hrv_cols = [c for c in df.columns if c.startswith("HRV_")]

# Kiểm tra nhanh
print("Số feature RRI:", len(rri_cols))
print("Số feature HRV:", len(hrv_cols))

# Cột meta
label_col = "Age_group"   # đổi nếu chị dùng tên khác
sex_col   = "Sex"         # 0/1
bmi_col   = "BMI"

# Lọc hàng hợp lệ (nếu có NaN)
df_clean = df.dropna(subset=rri_cols + hrv_cols + [label_col, sex_col, bmi_col])
print("Sau khi drop NaN:", df_clean.shape)


In [ ]:
# ==========================
# 3. TẠO 2 BỘ FEATURE:
#    - X1: RRI + HRV
#    - X2: RRI + HRV + Sex + BMI
# ==========================
X_rri_hrv = df_clean[rri_cols + hrv_cols].values
X_full    = df_clean[rri_cols + hrv_cols + [sex_col, bmi_col]].values

y = df_clean[label_col].values  # Age group

print("X_rri_hrv:", X_rri_hrv.shape)
print("X_full   :", X_full.shape)
print("y        :", y.shape)


In [ ]:
# ==========================
# 4. CHUẨN HÓA (SCALING)
# Lưu ý: đây chỉ là scaling phục vụ TRỰC QUAN HÓA,
# không bắt buộc giống pipeline train model.
# ==========================
scaler1 = StandardScaler()
scaler2 = StandardScaler()

X_rri_hrv_scaled = scaler1.fit_transform(X_rri_hrv)
X_full_scaled    = scaler2.fit_transform(X_full)

X_rri_hrv_scaled.shape, X_full_scaled.shape


In [ ]:
# ==========================
# 5. CHẠY UMAP CHO 2 TRƯỜNG HỢP
# ==========================
umap_model1 = umap.UMAP(
    n_components=2,
    n_neighbors=30,     # có thể thử 15, 50,...
    min_dist=0.1,
    metric="euclidean",
    random_state=42
)
umap_model2 = umap.UMAP(
    n_components=2,
    n_neighbors=30,
    min_dist=0.1,
    metric="euclidean",
    random_state=42
)

Z_rri_hrv = umap_model1.fit_transform(X_rri_hrv_scaled)  # (N, 2)
Z_full    = umap_model2.fit_transform(X_full_scaled)     # (N, 2)

Z_rri_hrv.shape, Z_full.shape


In [ ]:
# ==========================
# 6. HÀM VẼ SCATTER THEO AGE GROUP
# ==========================
import matplotlib.cm as cm

def plot_umap_2d(Z, y, title="UMAP projection"):
    """
    Z: (N, 2) embedding
    y: vector nhãn (Age group)
    """
    unique_labels = np.unique(y)
    # Chọn colormap với số màu = số lớp
    colors = cm.get_cmap("tab10", len(unique_labels))

    plt.figure(figsize=(6, 5))
    for idx, lab in enumerate(unique_labels):
        mask = (y == lab)
        plt.scatter(
            Z[mask, 0],
            Z[mask, 1],
            s=10,
            alpha=0.7,
            label=f"Age group {lab}",
            color=colors(idx)
        )
    plt.title(title)
    plt.xlabel("UMAP-1")
    plt.ylabel("UMAP-2")
    plt.legend(markerscale=2, fontsize=8)
    plt.tight_layout()
    plt.show()


In [ ]:
# ==========================
# 7. VẼ TRƯỚC & SAU KHI THÊM SEX + BMI
# ==========================

plot_umap_2d(Z_rri_hrv, y, title="UMAP – RRI + HRV")
plot_umap_2d(Z_full,    y, title="UMAP – RRI + HRV + Sex + BMI")
